In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [30]:
username = "052123"

In [2]:
# Spark Dynamic Allocation is off by default but CML Overrides that to true
# Disabling DA for demo purposes. We will use it later.

# Spark AQE is on by default in Spark 3.2+
# Disabling AQE for demo purposes. We will use it later.

spark = SparkSession\
    .builder\
    .appName("PythonSQL")\
    .config("spark.hadoop.fs.s3a.s3guard.ddb.region","us-east-2")\
    .config("spark.yarn.access.hadoopFileSystems","s3a://go01-demo")\
    .config("spark.sql.adaptive.enabled", "false")\
    .config("spark.dynamicAllocation.enabled", "false")\
    .getOrCreate()

Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco


In [3]:
spark.sparkContext.getConf().getAll()

[('spark.dynamicAllocation.enabled', 'false'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.ui.proxyRedirectUri',
  'https://spark-hgx2s40gc88b8mgh.ml-4c5feac0-3ec.go01-dem.ylcu-atmi.cloudera.site'),
 ('spark.hadoop.fs.s3a.s3guard.ddb.region', 'us-east-2'),
 ('spark.network.crypto.enabled', 'true'),
 ('spark.kubernetes.driver.pod.name', 'hgx2s40gc88b8mgh'),
 ('spark.kerberos.renewal.credentials', 'ccache'),
 ('spark.dynamicAllocation.maxExecutors', '49'),
 ('spark.eventLog.dir', 'file:///sparkeventlogs'),
 ('spark.hadoop.yarn.resourcemanager.principal', 'pauldefusco'),
 ('spark.kubernetes.driver.annotation.cluster-autoscaler.kubernetes.io/safe-to-evict',
  'false'),
 ('spark.ui.port', '20049'),
 ('spark.kubernetes.executor.annotation.cluster-autoscaler.kubernetes.io/safe-to-evict',
  'false'),
 ('spark.driver.memory', '3051m'),
 ('spark.io.encryption.enabled', 'true'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.bindAddress', '100.100.92.161'),
 ('spark.submit.depl

https://mageswaran1989.medium.com/spark-jargon-for-starters-af1fd8117ada#:~:text=A%20worker%20node%20can%20be,sufficient%20CPU%2C%20Memory%20and%20Storage.

https://www.linkedin.com/pulse/just-enough-spark-core-concepts-revisited-deepak-rajak/

#### Spark Basic Architecture
A cluster, or group of machines, pools the resources of many machines together allowing us to use all the cumulative resources as if they were one. Now a group of machines sitting somewhere alone is not powerful, you need a framework to coordinate work across them. Spark is a tailor-made engine exactly for this, managing and coordinating the execution of tasks on data across a cluster of computers. 

The cluster of machines that Spark will leverage to execute tasks will be managed by a cluster manager like Spark’s Standalone cluster manager, YARN - Yet Another Resource Negotiator, Kubernetes. We then submit Spark Applications to these cluster managers which will grant resources to our application so that we can complete our work. 

#### Spark Application
Spark Applications consist of a driver process and a set of executor processes. In the illustration we see above, our driver is on the left and four executors on the right.

#### Spark Worker Nodes
The worker nodes contain the executors which are responsible for actually carrying out the work that the driver assigns them. The Cluster Manager controls physical machines and allocates resources to the Spark Application. There can be multiple Spark Applications running on a cluster at the same time.



#### What is a JVM?
The JVM manages system memory and provides a portable execution environment for Java-based applications 

Technical definition: The JVM is the specification for a software program that executes code and provides the runtime environment for that code. 

Everyday definition: The JVM is how we run our Java programs. We configure the JVM's settings and then rely on it to manage program resources during execution. 

The Java Virtual Machine (JVM) is a program whose purpose is to execute other programs. 

The JVM has two primary functions: 

To allow Java programs to run on any device or operating system (known as the "Write once, run anywhere" principle)
To manage and optimize program memory 

#### What is a Driver?
The driver process runs your main() functions , sits on the node in the cluster and is responsible for 3 main things:

Maintaining information about the spark application. Its a heart of a spark application and maintains all the I information during the lifetime of the application.
Responding to user’s program or input.
Analyzing, distributing and scheduling work across the executors.

When a Spark program is triggered (e.g. a cell in this notebook performing Spark transformations and actions) the Driver converts the user program into tasks and after that it schedules the tasks on the executors.

Spark Application — -> Driver — -> List of Tasks — -> Scheduler — -> Executors

#### What is an Executor?
The executors are responsible for carrying out the work that the driver assigns them.

Executors are worker nodes’ processes in charge of running individual tasks in a given Spark job. They are launched at the beginning of a Spark application and typically run for the entire lifetime of an application. Once they have run the task they send the results to the driver. They also provide in-memory storage for RDDs that are cached by user programs through Block Manager.

When executors are started they register themselves with the driver and from so on they communicate directly. The workers are in charge of communicating the cluster manager the availability of their resources.

* Execute code assigned to it by the driver.
* Reporting the state of the computation on that executor back to driver.

#### What is Cores/Slots/Threads?
Spark parallelizes at two levels. One is the splitting the work among executors. The other is the slot. Each executor has a number of slots. Each slot can be assigned a Task. 

The JVM is naturally multithreaded, but a single JVM, such as our Driver, has a finite upper limit.
By creating Tasks, the Driver can assign units of work to Slots on each Executor for parallel execution.
Additionally, the Driver must also decide how to partition the data so that it can be distributed for parallel processing (see below).
Consequently, the Driver is assigning a Partition of data to each task - in this way each Task knows which piece of data it is to process.
Once started, each Task will fetch from the original data source (e.g. An Azure Storage Account) the Partition of data assigned to it.
You can set the number of task slots to a value two or three times (i.e. to a multiple of) the number of CPU cores. Although these task slots are often referred to as CPU cores in Spark, they’re implemented as threads that work on a physical core's thread and don’t need to correspond to the number of physical CPU cores on the machine (since different CPU manufacturer's can architect multi-threaded chips differently). 

In other words:

All processors of today have multiple cores (e.g. 1 CPU = 8 Cores)
Most processors of today are multi-threaded (e.g. 1 Core = 2 Threads, 8 cores = 16 Threads)
A Spark Task runs on a Slot. 1 Thread is capable of doing 1 Task at a time. To make use of all our threads on the CPU, we cleverly assign the number of Slots to correspond to a multiple of the number of Cores (which translates to multiple Threads).
By doing this, after the Driver breaks down a given command into Tasks and Partitions, which are tailor-made to fit our particular Cluster Configuration (say 4 nodes - 1 driver and 3 executors, 8 cores per node, 2 threads per core). By using our Clusters at maximum efficiency like this (utilizing all available threads), we can get our massive command executed as fast as possible (given our Cluster in this case, 3*8*2 Threads --> 48 Tasks, 48 Partitions - i.e. 1 Partition per Task)
If we don't do then even with a 100 executor cluster, the entire burden would go to 1 executor, and the other 99 will be sitting idle - i.e. slow execution.
if we foolishly assign 49 Tasks and 49 Partitions, the first pass would execute 48 Tasks in parallel across the executors cores (say in 10 minutes), then that 1 remaining Task in the next pass will execute on 1 core for another 10 minutes, while the rest of our 47 cores are sitting idle - meaning the whole job will take double the time at 20 minutes. This is obviously an inefficient use of our available resources, and could rather be fixed by setting the number of tasks/partitions to a multiple of the number of cores we have (in this setup - 48, 96 etc).

#### What is a Partition?
In order to allow every executor to perform work in parallel, Spark breaks up the data into chunks, called partitions.

A partition is a collection of rows that sit on one physical machine in our cluster. A DataFrame’s partitions represent how the data is physically distributed across your cluster of machines during execution:

If you have one partition, Spark will only have a parallelism of one, even if you have thousands of executors. 
If you have many partitions, but only one executor, Spark will still only have a parallelism of one because there is only one computation resource. 
An important thing to note is that with DataFrames, we do not (for the most part) manipulate partitions manually (on an individual basis). We simply specify high level transformations of data in the physical partitions and Spark determines how this work will actually execute on the cluster.

#### What happens when you launch a Spark Session
When you create the SparkContext, each worker starts an executor. This is a separate process (JVM), and it loads your jar too. The executors connect back to your driver program. Now the driver can send them commands, like flatMap, map and reduceByKey in your example. When the driver quits, the executors shut down.

RDDs are sort of like big arrays that are split into partitions, and each executor can hold some of these partitions.

A task is a command sent from the driver to an executor by serializing your Function object. The executor deserializes the command (this is possible because it has loaded your jar), and executes it on a partition.

#### What is a DAG?
Directed Acyclic Graph ( DAG ) in Apache Spark is a set of Vertices and Edges, where vertices represent the RDDs and the edges represent the Operation to be applied on RDDs.

DAGScheduler is the scheduling layer of Apache Spark that implements stage-oriented scheduling. It transforms a logical execution plan to a physical execution plan (using stages).

After an action has been called, SparkContext hands over a logical plan to DAGScheduler that it in turn translates to a set of stages that are submitted as a set of tasks for execution.

The fundamental concepts of DAGScheduler are jobs and stages that it tracks through internal registries and counters.

#### What is a Job?
A Job is a sequence of stages, triggered by an action such as count(), collect(), read() or write(). 

Each parallelized action is referred to as a Job.
The results of each Job (parallelized/distributed action) is returned to the Driver from the Executor.
Depending on the work required, multiple Jobs will be required.

#### What is a Stage?
Each job that gets divided into smaller sets of tasks is a stage.

A Stage is a sequence of Tasks that can all be run together - i.e. in parallel - without a shuffle. For example: using ".read" to read a file from disk, then runnning ".filter" can be done without a shuffle, so it can fit in a single stage. The number of Tasks in a Stage also depends upon the number of Partitions your datasets have.

#### What is a Task?
A task is a unit of work that is sent to the executor. Each stage has some tasks, one task per partition. The same task is done over different partitions of the RDD.

In the example of Stages above, each Step is a Task.



#### What is a Caching?
In applications that reuse the same datasets over and over, one of the most useful optimizations is caching. Caching will place a DataFrame or table into temporary storage across the executors in your cluster and make subsequent reads faster. 



#### What is a Shuffling?
A Shuffle refers to an operation where data is re-partitioned across a Cluster - i.e. when data needs to move between executors.

join and any operation that ends with ByKey will trigger a Shuffle. It is a costly operation because a lot of data can be sent via the network.

For example, to group by color, it will serve us best if...

All the reds are in one partitions
All the blues are in a second partition
All the greens are in a third
From there we can easily sum/count/average all of the reds, blues, and greens.

#### What is a Partitioning?
A Partition is a logical chunk of your DataFrame

Data is split into Partitions so that each Executor can operate on a single part, enabling parallelization.

It can be processed by a single Executor core/thread.

For example: If you have 4 data partitions and you have 4 executor cores/threads, you can process everything in parallel, in a single pass.

#### Transformations

In Spark, the core data structures are immutable meaning they cannot be changed once created. In order to "change" a DataFrame you will have to instruct Spark how you would like to modify the DataFrame you have into the one that you want. These instructions are called transformations. Let’s perform a simple transformation to find all even numbers in our currentDataFrame. Examples – Select, Filter, GroupBy, Join, Union, Partition etc

Actions

Transformations allow us to build up our logical transformation plan. To trigger the computation, we run an action. An action instructs Spark to compute a result from a series of transformations. The simplest action is count which gives us the total number of records in the DataFrame.

#### Narrow Transformations Vs Wide Transformations
There are two types of transformations: Narrow and Wide.

For narrow transformations, the data required to compute the records in a single partition reside in at most one partition of the parent dataset. 

Examples include:

filter(..)
drop(..)
coalesce()
For wide transformations, the data required to compute the records in a single partition may reside in many partitions of the parent dataset. 

Examples include:

distinct()
groupBy(..).sum()
repartition(n)
Remember, spark partitions are collections of rows that sit on physical machines in the cluster. Narrow transformations mean that work can be computed and reported back to the executor without changing the way data is partitioned over the system. Wide transformations require that data be redistributed over the system. This is called a shuffle. 

Shuffles are triggered when data needs to move between executors. 

### Application Execution Flow
With this in mind, when you submit an application to the cluster with spark-submit this is what happens internally:

An application starts and instantiates a SparkContext/SparkSession instance (and it is only then when you can call the application a driver).
The driver program ask for resources to the cluster manager to launch executors.
The cluster manager launches executors.
The driver process runs through the user application. Depending on the actions and transformations over RDDs task are sent to executors.
Executors run the tasks and save the results.
If any worker crashes, its tasks will be sent to different executors to be processed again.

Spark automatically deals with failed or slow machines by re-executing failed or slow tasks. For example, if the node running a partition of a map() operation crashes, Spark will rerun it on another node; and even if the node does not crash but is simply much slower than other nodes, Spark can preemptively launch a “speculative” copy of the task on another node, and take its result if that finishes.

#### Dataframes vs Datasets vs RDDs

Initially, in 2011 in they came up with the concept of RDDs, then in 2013 with Dataframes and later in 2015 with the concept of Datasets. None of them has been depreciated, we can still use all of them.

#### What are RDDs?
RDDs or Resilient Distributed Datasets is the fundamental data structure of the Spark. It is the collection of objects which is capable of storing the data partitioned across the multiple nodes of the cluster and also allows them to do processing in parallel.

It is fault-tolerant if you perform multiple transformations on the RDD and then due to any reason any node fails. The RDD, in that case, is capable of recovering automatically.

#### What are Dataframes?
It was introduced first in Spark version 1.3 to overcome the limitations of the Spark RDD. Spark Dataframes are the distributed collection of the data points, but here, the data is organized into the named columns. They allow developers to debug the code during the runtime which was not allowed with the RDDs.

Dataframes can read and write the data into various formats like CSV, JSON, AVRO, HDFS, and HIVE tables. It is already optimized to process large datasets for most of the pre-processing tasks so that we do not need to write complex functions on our own.

It uses a catalyst optimizer for optimization purposes.

#### What are Datasets?
Spark Datasets is an extension of Dataframes API with the benefits of both RDDs and the Datasets. It is fast as well as provides a type-safe interface. Type safety means that the compiler will validate the data types of all the columns in the dataset while compilation only and will throw an error if there is any mismatch in the data types.

Users of RDD will find it somewhat similar to code but it is faster than RDDs. It can efficiently process both structured and unstructured data.

We cannot create Spark Datasets in Python yet. The dataset API is available only in Scala and Java only

Q1. What is difference between RDD and DataFrame?
A. RDD (Resilient Distributed Dataset) is the fundamental data structure in Apache Spark, representing an immutable distributed collection of objects. It offers low-level operations and lacks optimization benefits provided by higher-level abstractions.
DataFrames, on the other hand, are higher-level abstractions built on top of RDDs. They provide structured and optimized distributed data processing with a schema, supporting SQL-like queries, and various optimizations for better performance.

Q2. When should I use RDD over DataFrame?
A. RDDs are useful in scenarios where you require low-level control over data and need to perform complex custom transformations or need to access RDD-specific operations not available in DataFrames. Additionally, RDDs are suitable when working with unstructured data or when integrating with non-Spark libraries that expect RDDs as input.

### FAQ's

Does every worker instance hold an executor for specific application (which manages storage, task) or one worker node holds one executor?

Workers hold many executors, for many applications. One application has executors on many workers

A worker node can be holding multiple executors (processes) if it has sufficient CPU, Memory and Storage.

### Working with More Data

In [25]:
from pyspark.sql.types import LongType, IntegerType, StringType

import dbldatagen as dg

INFO: Version : VersionInfo(major='0', minor='2', patch='1', release='', build='')


In [27]:
def generate_df(row_count = 100000, unique_vals=100000):

    #shuffle_partitions_requested = 8
    
    #spark.conf.set("spark.sql.shuffle.partitions", shuffle_partitions_requested)

    country_codes = [
        "CN", "US", "FR", "CA", "IN", "JM", "IE", "PK", "GB", "IL", "AU", 
        "SG", "ES", "GE", "MX", "ET", "SA", "LB", "NL", "IT"
    ]
    #country_weights = [
    #    1300, 365, 67, 38, 1300, 3, 7, 212, 67, 9, 25, 6, 47, 83, 
    #    126, 109, 58, 8, 17,
    #]

    manufacturers = [
        "Delta corp", "Xyzzy Inc.", "Lakehouse Ltd", "Acme Corp", "Embanks Devices",
    ]

    lines = ["delta", "xyzzy", "lakehouse", "gadget", "droid"]

    testDataSpec = (
        dg.DataGenerator(spark, name="device_data_set", rows=row_count) #,partitions=partitions_num)
        .withIdOutput()
        # we'll use hash of the base field to generate the ids to
        # avoid a simple incrementing sequence
        .withColumn("internal_device_id", "long", minValue=0x1000000000000, 
                    uniqueValues=unique_vals, omit=True, baseColumnType="hash",
        )
        # note for format strings, we must use "%lx" not "%x" as the
        # underlying value is a long
        .withColumn(
            "device_id", "string", format="0x%013x", baseColumn="internal_device_id"
        )
        # the device / user attributes will be the same for the same device id
        # so lets use the internal device id as the base column for these attribute
        .withColumn("country", "string", values=country_codes, #weights=country_weights, 
                    baseColumn="internal_device_id")
        .withColumn("manufacturer", "string", values=manufacturers, 
                    baseColumn="internal_device_id", )
        # use omit = True if you don't want a column to appear in the final output
        # but just want to use it as part of generation of another column
        .withColumn("line", "string", values=lines, baseColumn="manufacturer", 
                    baseColumnType="hash", omit=True )
        .withColumn("model_ser", "integer", minValue=1, maxValue=11, baseColumn="device_id", 
                    baseColumnType="hash", omit=True, )
        .withColumn("model_line", "string", expr="concat(line, '#', model_ser)", 
                    baseColumn=["line", "model_ser"] )
        .withColumn("event_type", "string", 
                    values=["activation", "deactivation", "plan change", "telecoms activity", 
                            "internet activity", "device error", ],
                    random=True)
        .withColumn("event_ts", "timestamp", begin="2020-01-01 01:00:00", 
                    end="2020-12-31 23:59:00", 
                    interval="1 minute", random=True )
    )

    dfTestData = testDataSpec.build()

    display(dfTestData)
    
    return dfTestData

When you are running Spark application in yarn or any cluster manager, the default length/size of partitions RDD/DataFrame/Dataset are created with the total number of cores on all executor nodes. 

Spark automatically sets the number of “map” tasks to run on each file according to its size (though you can control it through optional parameters to SparkContext.textFile, etc), and for distributed “reduce” operations, such as groupByKey and reduceByKey, it uses the largest parent RDD’s number of partitions.

In [28]:
df = generate_df(row_count = 100000, unique_vals=100000)

DataFrame[id: bigint, device_id: string, country: string, manufacturer: string, model_line: string, event_type: string, event_ts: timestamp]

In [29]:
df.rdd.getNumPartitions()

10

In [31]:
df.write.mode("overwrite").saveAsTable('SPARK.IOT_DATA_{}'.format(username), format="parquet") #partitionBy()

In [35]:
spark.conf.get("spark.sql.warehouse.dir")

's3a://go01-demo/warehouse/tablespace/external/hive'

In [41]:
spark.conf.get("spark.kubernetes.container.image")

'docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.7-standard:2022.11.1-b2'

Turn AQE and DA off. Run spark jobs.

Turn DA On and do section on it

Turn AQE On and do section on it

#### Spark Challenges

Spark Merge Into Workaround

Spark Schema Evolution Workaround

Spark Partition Evolution Workaround